# The Transformer Architecture
:label:`sec_transformer`


We have compared CNNs, RNNs, and self-attention in
:numref:`subsec_cnn-rnn-self-attention`.
Notably, self-attention
enjoys both parallel computation and
the shortest maximum path length.
Therefore naturally,
it is appealing to design deep architectures
by using self-attention.
Unlike earlier self-attention models
that still rely on RNNs for input representations :cite:`Cheng.Dong.Lapata.2016,Lin.Feng.Santos.ea.2017,Paulus.Xiong.Socher.2017`,
the Transformer model
is solely based on attention mechanisms
without any convolutional or recurrent layer :cite:`Vaswani.Shazeer.Parmar.ea.2017`.
Though originally proposed
for sequence to sequence learning on text data,
Transformers have been
pervasive in a wide range of
modern deep learning applications,
such as in areas of language, vision, speech, and reinforcement learning.

## Model

As an instance of the encoder-decoder
architecture,
the overall architecture of
the Transformer
is presented in :numref:`fig_transformer`.
As we can see,
the Transformer is composed of an encoder and a decoder.
Different from
Bahdanau attention
for sequence to sequence learning
in :numref:`fig_s2s_attention_details`,
the input (source) and output (target)
sequence embeddings
are added with positional encoding
before being fed into
the encoder and the decoder
that stack modules based on self-attention.

![The Transformer architecture.](https://d2l.ai/_images/transformer.svg)
:width:`400px`
:label:`fig_transformer`


Now we provide an overview of the
Transformer architecture in :numref:`fig_transformer`.
On a high level,
the Transformer encoder is a stack of multiple identical layers,
where each layer
has two sublayers (either is denoted as $\mathrm{sublayer}$).
The first
is a multi-head self-attention pooling
and the second is a positionwise feed-forward network.
Specifically,
in the encoder self-attention,
queries, keys, and values are all from the
outputs of the previous encoder layer.
Inspired by the ResNet design in :numref:`sec_resnet`,
a residual connection is employed
around both sublayers.
In the Transformer,
for any input $\mathbf{x} \in \mathbb{R}^d$ at any position of the sequence,
we require that $\mathrm{sublayer}(\mathbf{x}) \in \mathbb{R}^d$ so that
the residual connection $\mathbf{x} + \mathrm{sublayer}(\mathbf{x}) \in \mathbb{R}^d$ is feasible.
This addition from the residual connection is immediately
followed by layer normalization :cite:`Ba.Kiros.Hinton.2016`.
As a result, the Transformer encoder outputs a $d$-dimensional vector representation 
for each position of the input sequence.

The Transformer decoder is also a stack of multiple identical layers 
with residual connections and layer normalizations.
Besides the two sublayers described in
the encoder, the decoder inserts
a third sublayer, known as
the encoder-decoder attention,
between these two.
In the encoder-decoder attention,
queries are from the
outputs of the previous decoder layer,
and the keys and values are
from the Transformer encoder outputs.
In the decoder self-attention,
queries, keys, and values are all from the
outputs of the previous decoder layer.
However, each position in the decoder is
allowed to only attend to all positions in the decoder
up to that position.
This *masked* attention
preserves the auto-regressive property,
ensuring that the prediction only depends 
on those output tokens that have been generated.


We have already described and implemented
multi-head attention based on scaled dot-products
in :numref:`sec_multihead-attention`
and positional encoding in :numref:`subsec_positional-encoding`.
In the following, we will implement
the rest of the Transformer model.

In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.Vocabulary
import ai.djl.modality.nlp.embedding.TrainableWordEmbedding
import jp.live.ugai.d2j.timemachine.RNNModelScratch
import jp.live.ugai.d2j.timemachine.TimeMachine.trainCh8
import jp.live.ugai.d2j.timemachine.TimeMachineDataset
import jp.live.ugai.d2j.timemachine.Vocab
import jp.live.ugai.d2j.RNNModel
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Accumulator
import jp.live.ugai.d2j.util.Training
import jp.live.ugai.d2j.util.TrainingChapter9
import jp.live.ugai.d2j.lstm.Decoder
import jp.live.ugai.d2j.lstm.Encoder
import jp.live.ugai.d2j.lstm.EncoderDecoder
import jp.live.ugai.d2j.util.NMT
import jp.live.ugai.d2j.attention.MultiHeadAttention
import jp.live.ugai.d2j.PositionalEncoding
import java.util.Locale
import kotlin.random.Random
import kotlin.collections.List
import kotlin.collections.Map
import kotlin.Pair

In [2]:
val manager = NDManager.newBaseManager()
val ps = ParameterStore(manager, false)

## [**Positionwise Feed-Forward Networks**]
:label:`subsec_positionwise-ffn`

The positionwise feed-forward network transforms
the representation at all the sequence positions
using the same MLP.
This is why we call it *positionwise*.
In the implementation below,
the input `X` with shape
(batch size, number of time steps or sequence length in tokens,
number of hidden units or feature dimension)
will be transformed by a two-layer MLP into
an output tensor of shape
(batch size, number of time steps, `ffn_num_outputs`).

In [3]:

fun positionWiseFFN(ffn_num_hiddens: Long, ffn_num_outputs: Long) : AbstractBlock {
    val net = SequentialBlock()
    net.add(Linear.builder().setUnits(ffn_num_hiddens).build())
    net.add(Activation::relu)
    net.add(Linear.builder().setUnits(ffn_num_outputs).build());
//    net.setInitializer(NormalInitializer(), Parameter.Type.WEIGHT)
    return net
}

The following example
shows that [**the innermost dimension
of a tensor changes**] to
the number of outputs in
the positionwise feed-forward network.
Since the same MLP transforms
at all the positions,
when the inputs at all these positions are the same,
their outputs are also identical.

In [4]:
val ffn = positionWiseFFN(4, 8)
ffn.initialize(manager, DataType.FLOAT32, Shape(2,3,4))
ffn.forward(ps, NDList(manager.ones(Shape(2,3,4))), false)[0][0]

ND: (3, 8) cpu() float32
[[ 1.0967, -3.0072, -0.3658, -0.6362,  2.8636,  3.2177,  5.0021, -2.5833],
 [ 1.0967, -3.0072, -0.3658, -0.6362,  2.8636,  3.2177,  5.0021, -2.5833],
 [ 1.0967, -3.0072, -0.3658, -0.6362,  2.8636,  3.2177,  5.0021, -2.5833],
]


## Residual Connection and Layer Normalization

Now let's focus on the "add & norm" component in :numref:`fig_transformer`.
As we described at the beginning of this section,
this is a residual connection immediately
followed by layer normalization.
Both are key to effective deep architectures.

In :numref:`sec_batch_norm`,
we explained how batch normalization
recenters and rescales across the examples within
a minibatch.
As discussed in :numref:`subsec_layer-normalization-in-bn`,
layer normalization is the same as batch normalization
except that the former
normalizes across the feature dimension,
thus enjoying benefits of scale independence and batch size independence.
Despite its pervasive applications
in computer vision,
batch normalization
is usually empirically
less effective than layer normalization
in natural language processing
tasks, whose inputs are often
variable-length sequences.

The following code snippet
[**compares the normalization across different dimensions
by layer normalization and batch normalization**].


In [5]:
val ln = LayerNorm.builder().build()
ln.initialize(manager, DataType.FLOAT32, Shape(2,2))
val bn = BatchNorm.builder().build()
bn.initialize(manager, DataType.FLOAT32, Shape(2,2))
val X = manager.create(floatArrayOf(1f,2f,2f,3f)).reshape(Shape(2,2))
print("LayerNorm: ")
println(ln.forward(ps, NDList(X), false)[0])
print("BatchNorm: ")
println(bn.forward(ps, NDList(X), false)[0])

LayerNorm: ND: (2, 2) cpu() float32
[[-1.    ,  1.    ],
 [-1.    ,  1.    ],
]

BatchNorm: ND: (2, 2) cpu() float32
[[1.    , 2.    ],
 [2.    , 3.    ],
]



Now we can implement the `AddNorm` class
[**using a residual connection followed by layer normalization**].
Dropout is also applied for regularization.


In [6]:
class AddNorm(rate: Float) : AbstractBlock() {
    val dropout = Dropout.builder().optRate(rate).build()
    val ln = LayerNorm.builder().build()

    init {
        addChildBlock("dropout", dropout)
        addChildBlock("layerNorm", ln)
    }

    override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        val x = inputs[0]
        val y = inputs[1]
        val dropoutResult = dropout.forward(ps, NDList(y), training, params).singletonOrThrow()
        val result = ln.forward(ps, NDList(dropoutResult.add(x)), training, params)
        return result
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        return arrayOf<Shape>()
    }

    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        val sub = manager.newSubManager()
            dropout.initialize(manager, dataType, *inputShapes)
            ln.initialize(manager, dataType, *inputShapes)
        sub.close()
    }
}

The residual connection requires that
the two inputs are of the same shape
so that [**the output tensor also has the same shape after the addition operation**].


In [7]:
val addNorm = AddNorm(0.5f)
addNorm.initialize(manager, DataType.FLOAT32, Shape(1,4))
addNorm.forward(ps, NDList(manager.ones(Shape(2,3,4)), manager.ones(Shape(2,3,4))), false)[0].shapeEquals(manager.ones(Shape(2,3,4)))

true

## Encoder
:label:`subsec_transformer-encoder`

With all the essential components to assemble
the Transformer encoder,
let's start by
implementing [**a single layer within the encoder**].
The following `TransformerEncoderBlock` class
contains two sublayers: multi-head self-attention and positionwise feed-forward networks,
where a residual connection followed by layer normalization is employed
around both sublayers.

In [8]:
class TransformerEncoderBlock(numHiddens: Int, ffnNumHiddens: Long, numHeads: Int , dropout: Float,
                 useBias: Boolean=false) : AbstractBlock() {
    val attention = MultiHeadAttention(numHiddens, numHeads, dropout, useBias)
    val addnorm1 = AddNorm(dropout)
    val ffn = positionWiseFFN(ffnNumHiddens, numHiddens.toLong())
    val addnorm2 = AddNorm(dropout)
    init {
        addChildBlock("attention", attention)
        addChildBlock("addnorm1", addnorm1)
        addChildBlock("ffn", ffn)
        addChildBlock("addnorm2", addnorm2)
    }

    override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        val x = inputs[0]
        val validLens = inputs[1]
        val y = addnorm1.forward(ps, NDList(x, attention.forward(ps, NDList(x,x,x, validLens), training, params).singletonOrThrow()), training, params)
        val ret = addnorm2.forward(ps, NDList(y.singletonOrThrow(), ffn.forward(ps, y, training, params).singletonOrThrow()), training, params)
        return ret
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        return arrayOf<Shape>()
    }

    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        val shapes = arrayOf(inputShapes[0], inputShapes[0], inputShapes[0], inputShapes[1])
            attention.initialize(manager, dataType, *shapes)
            addnorm1.initialize(manager, dataType, inputShapes[0])
            ffn.initialize(manager, dataType, inputShapes[0])
            addnorm2.initialize(manager, dataType, inputShapes[0])
    }
}

As we can see,
[**any layer in the Transformer encoder
does not change the shape of its input.**]


In [9]:
val X = manager.ones(Shape(2, 100, 24))
val validLens = manager.create(floatArrayOf(3f,2f))
val encoderBlock = TransformerEncoderBlock(24,48,8, 0.5f)
encoderBlock.initialize(manager, DataType.FLOAT32, X.shape, validLens.shape)
encoderBlock.forward(ps, NDList(X, validLens), false)

NDList size: 1
0 : (2, 100, 24) float32


In the following [**Transformer encoder**] implementation,
we stack `num_blks` instances of the above `TransformerEncoderBlock` classes.
Since we use the fixed positional encoding
whose values are always between -1 and 1,
we multiply values of the learnable input embeddings
by the square root of the embedding dimension
to rescale before summing up the input embedding and the positional encoding.

In [10]:
class TransformerEncoder(vocabSize: Int, val numHiddens: Int, ffnNumHiddens: Long,
                 numHeads: Long, numBlks: Int, dropout: Float, useBias: Boolean = false): Encoder() {

    private val embedding: TrainableWordEmbedding
    val posEncoding = PositionalEncoding(numHiddens, dropout, 1000, manager)
    val blks = mutableListOf<TransformerEncoderBlock>()
    val attentionWeights = Array<NDArray?>(numBlks){ null }

    /* The RNN encoder for sequence to sequence learning. */
    init {
        val list: List<String> = (0 until vocabSize).map { it.toString() }
        val vocab: Vocabulary = DefaultVocabulary(list)
        // Embedding layer
        embedding = TrainableWordEmbedding.builder()
            .optNumEmbeddings(vocabSize)
            .setEmbeddingSize(numHiddens)
            .setVocabulary(vocab)
            .build()
        addChildBlock("embedding", embedding)
        repeat(numBlks) {
            blks.add(TransformerEncoderBlock(numHiddens, ffnNumHiddens, numHeads.toInt(), dropout, useBias))
        }
    }

    override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        var X = inputs[0]
        val validLens = inputs[1]
        val emb = embedding.forward(ps, NDList(X), training, params).singletonOrThrow().mul(Math.sqrt(numHiddens.toDouble()))
        X = posEncoding.forward(ps, NDList(emb), training, params).singletonOrThrow()
        for(i in 0 until blks.size) {
            X = blks[i].forward(ps, NDList(X, validLens), training, params).singletonOrThrow()
            attentionWeights[i] = blks[i].attention.attention.attentionWeights
        }
        return NDList(X)    
    }
    
    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        embedding.initialize(manager, dataType, *inputShapes)
        for(blk in blks) {
            blk.initialize(manager, dataType, inputShapes[0].add(numHiddens.toLong()), inputShapes[1])
        }
    }
}

Below we specify hyperparameters to [**create a two-layer Transformer encoder**].
The shape of the Transformer encoder output
is (batch size, number of time steps, `num_hiddens`).


In [11]:
val encoder = TransformerEncoder(200, 24, 48, 8, 2, 0.5f)
encoder.initialize(manager, DataType.FLOAT32, Shape(2,100), validLens.shape)
encoder.forward(ps, NDList(manager.ones(Shape(2, 100)), validLens), false)

NDList size: 1
0 : (2, 100, 24) float32


## Decoder

As shown in :numref:`fig_transformer`,
[**the Transformer decoder
is composed of multiple identical layers**].
Each layer is implemented in the following
`TransformerDecoderBlock` class,
which contains three sublayers:
decoder self-attention,
encoder-decoder attention,
and positionwise feed-forward networks.
These sublayers employ
a residual connection around them
followed by layer normalization.


As we described earlier in this section,
in the masked multi-head decoder self-attention
(the first sublayer),
queries, keys, and values
all come from the outputs of the previous decoder layer.
When training sequence-to-sequence models,
tokens at all the positions (time steps)
of the output sequence
are known.
However,
during prediction
the output sequence is generated token by token;
thus,
at any decoder time step
only the generated tokens
can be used in the decoder self-attention.
To preserve auto-regression in the decoder,
its masked self-attention
specifies  `dec_valid_lens` so that
any query
only attends to
all positions in the decoder
up to the query position.

In [26]:
class TransformerDecoderBlock(val numHiddens: Int, ffnNumHiddens: Long,
                 numHeads: Long, dropout: Float, _i: Int): AbstractBlock() {
    val i = _i
    val attention1 = MultiHeadAttention(numHiddens, numHeads.toInt(), dropout, false)
    val addnorm1 = AddNorm(dropout)
    val attention2 = MultiHeadAttention(numHiddens, numHeads.toInt(), dropout, false)
    val addnorm2 = AddNorm(dropout)
    val ffn = positionWiseFFN(ffnNumHiddens, numHiddens.toLong())
    val addnorm3 = AddNorm(dropout)
    
    init {
        addChildBlock("attention1", attention1)
        addChildBlock("addnorm1", addnorm1)
        addChildBlock("attention2", attention2)
        addChildBlock("addnorm2", addnorm2)
        addChildBlock("ffn", ffn)
        addChildBlock("addnorm3", addnorm3)
    }

    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        val shapes = arrayOf(inputShapes[0], inputShapes[0], inputShapes[0], inputShapes[1])
            attention1.initialize(manager, dataType, *shapes)
            addnorm1.initialize(manager, dataType, inputShapes[0])
            attention2.initialize(manager, dataType, *shapes)
            addnorm2.initialize(manager, dataType, inputShapes[0])
            ffn.initialize(manager, dataType, inputShapes[0])
            addnorm3.initialize(manager, dataType, inputShapes[0])
    }
        override fun forwardInternal(
        ps: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
            val X = inputs[0]
            val encOutputs =  inputs[1]
            val envValidLens = inputs[2]
//        # During training, all the tokens of any output sequence are processed
//        # at the same time, so state[2][self.i] is None as initialized. When
//        # decoding any output sequence token by token during prediction,
//        # state[2][self.i] contains representations of the decoded output at
//        # the i-th block up to the current time step
//        if state[2][self.i] is None:
//            key_values = X
//        else:
            var keyValues : NDArray 
            println(inputs.size)
            if(inputs.size < 4) {
                keyValues = inputs[0]
            } else {
                keyValues = inputs[3].get(i.toLong()).concat(X, 1)
                inputs[3].set(NDIndex(i.toLong()), keyValues)
            }

        var decValidLens: NDArray?
        if (training) {
            val batchSize = X.shape[0]
            val numSteps = X.shape[1]
            //  Shape of dec_valid_lens: (batch_size, num_steps), where every
            //  row is [1, 2, ..., num_steps]
            decValidLens = manager.arange(1f, (numSteps+1).toFloat()).repeat(batchSize.toInt(),1)
        } else {
            decValidLens = null
        }
//        # Self-attention
        val X2 = attention1.forward(ps, NDList(X, keyValues, keyValues, decValidLens), training)
        val Y = addnorm1.forward(ps, NDList(X, X2.head()), training)
//        # Encoder-decoder attention. Shape of enc_outputs:
//        # (batch_size, num_steps, num_hiddens)
        val Y2 = attention2.forward(ps, NDList(Y.head(), encOutputs, encOutputs, envValidLens), training)
        val Z = addnorm2.forward(ps, NDList(Y.head(), Y2.head()), training)
        return NDList(addnorm3.forward(ps, NDList(Z.head(), ffn.forward(ps, NDList(Z), training).head()), training).head(), encOutputs, envValidLens, keyValues)
        }
        
    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        return arrayOf<Shape>()
    }

}



To facilitate scaled dot-product operations
in the encoder-decoder attention
and addition operations in the residual connections,
[**the feature dimension (`num_hiddens`) of the decoder is
the same as that of the encoder.**]


In [29]:
val decoderBlk = TransformerDecoderBlock(24, 48, 8, 0.5f, 0)
val X = manager.ones(Shape(2, 100, 24))
decoderBlk.initialize(manager, DataType.FLOAT32, X.shape, validLens.shape)
val state = encoderBlock.forward(ps, NDList(X, validLens, validLens), false)
println(state)
decoderBlk.forward(ps, NDList(X, validLens, validLens), true)

NDList size: 1
0 : (2, 100, 24) float32

3


SequenceLength must be of shape [batchSize]
java.lang.IllegalArgumentException: SequenceLength must be of shape [batchSize]
	at ai.djl.mxnet.engine.MxNDArray.sequenceMask(MxNDArray.java:389)
	at jp.live.ugai.d2j.attention.Chap10Utils.maskedSoftmax(Chap10Utils.kt:47)
	at jp.live.ugai.d2j.attention.DotProductAttention.forwardInternal(DotProductAttention.kt:150)
	at ai.djl.nn.AbstractBaseBlock.forward(AbstractBaseBlock.java:77)
	at jp.live.ugai.d2j.attention.MultiHeadAttention.forwardInternal(MultiHeadAttention.kt:63)
	at ai.djl.nn.AbstractBaseBlock.forward(AbstractBaseBlock.java:77)
	at ai.djl.nn.Block.forward(Block.java:123)
	at Line_336$TransformerDecoderBlock.forwardInternal(Line_336.jupyter-kts:66)
	at ai.djl.nn.AbstractBaseBlock.forward(AbstractBaseBlock.java:77)
	at ai.djl.nn.Block.forward(Block.java:123)
	at Line_368.<init>(Line_368.jupyter-kts:6)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeC

Now we [**construct the entire Transformer decoder**]
composed of `num_blks` instances of `TransformerDecoderBlock`.
In the end,
a fully connected layer computes the prediction
for all the `vocab_size` possible output tokens.
Both of the decoder self-attention weights
and the encoder-decoder attention weights
are stored for later visualization.


## [**Training**]

Let's instantiate an encoder-decoder model
by following the Transformer architecture.
Here we specify that
both the Transformer encoder and the Transformer decoder
have 2 layers using 4-head attention.
Similar to :numref:`sec_seq2seq_training`,
we train the Transformer model
for sequence to sequence learning on the English-French machine translation dataset.

After training,
we use the Transformer model
to [**translate a few English sentences**] into French and compute their BLEU scores.


Let's [**visualize the Transformer attention weights**] when translating the last English sentence into French.
The shape of the encoder self-attention weights
is (number of encoder layers, number of attention heads, `num_steps` or number of queries, `num_steps` or number of key-value pairs).


In the encoder self-attention,
both queries and keys come from the same input sequence.
Since padding tokens do not carry meaning,
with specified valid length of the input sequence,
no query attends to positions of padding tokens.
In the following,
two layers of multi-head attention weights
are presented row by row.
Each head independently attends
based on a separate representation subspaces of queries, keys, and values.

[**To visualize both the decoder self-attention weights and the encoder-decoder attention weights,
we need more data manipulations.**]
For example,
we fill the masked attention weights with zero.
Note that
the decoder self-attention weights
and the encoder-decoder attention weights
both have the same queries:
the beginning-of-sequence token followed by
the output tokens and possibly
end-of-sequence tokens.

Due to the auto-regressive property of the decoder self-attention,
no query attends to key-value pairs after the query position.

Similar to the case in the encoder self-attention,
via the specified valid length of the input sequence,
[**no query from the output sequence
attends to those padding tokens from the input sequence.**]

Although the Transformer architecture
was originally proposed for sequence-to-sequence learning,
as we will discover later in the book,
either the Transformer encoder
or the Transformer decoder
is often individually used
for different deep learning tasks.


## Summary

The Transformer is an instance of the encoder-decoder architecture, 
though either the encoder or the decoder can be used individually in practice.
In the Transformer architecture, multi-head self-attention is used 
for representing the input sequence and the output sequence, 
though the decoder has to preserve the auto-regressive property via a masked version.
Both the residual connections and the layer normalization in the Transformer
are important for training a very deep model.
The positionwise feed-forward network in the Transformer model 
transforms the representation at all the sequence positions using the same MLP.

## Exercises

1. Train a deeper Transformer in the experiments. How does it affect the training speed and the translation performance?
1. Is it a good idea to replace scaled dot-product attention with additive attention in the Transformer? Why?
1. For language modeling, should we use the Transformer encoder, decoder, or both? How to design this method?
1. What can be challenges to Transformers if input sequences are very long? Why?
1. How to improve computational and memory efficiency of Transformers? Hint: you may refer to the survey paper by :citet:`Tay.Dehghani.Bahri.ea.2020`.
